In [119]:
!pip install fuzzywuzzy
!pip install sklearn
!pip install pandas
!pip install numpy

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn
import numpy as np
import time
from enum import Enum

!pip install multiprocess
import multiprocess
from multiprocess import Pool, Process

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanshkedia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [180]:
start = time.time()
# path = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/final(Cleaned and Analysed)_h01-20200818-153021.csv'
path = '/Users/priyanshkedia/Documents/Machine Learning/Twitter-Analysis/output_data/Tue Aug 18/quote_rt.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python')

#path = '/home/manikya_varshney/Documents/Python/Yale/Final Data/final(Cleaned and Analysed via gzip)_August18th.gzip'
#data = pd.read_parquet(path)

data_Imp = data.copy()
data_Ada = data.copy()
data_Ne = data.copy()
data_Sd = data.copy()
data_Purp = data.copy()
data_Pe = data.copy()
end = time.time()
print("Read parquet with pandas: ",(end-start),"sec")

Read parquet with pandas:  1.0906710624694824 sec


In [181]:
#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

In [182]:
# Stemming
def stem_sentences(keywords):
    ps = PorterStemmer()
    keywords = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords]
    keywords = [" ".join(sentence) for sentence in keywords]
    return keywords

In [183]:
# Cleaning the keywords
def clean_keywords(keywords):
    for i in range(len(keywords)): 
        keywords[i] = keywords[i].lower()

    #Remove punctuations   
    for i in range(len(keywords)): 
        keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

    #More cleaning
    for i in range(len(keywords)): 
        keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')
    
    keywords = remove_stopwords(keywords)

    keywords = stem_sentences(keywords)
    
    return keywords

In [184]:
# def fuzzy_Imp(row):
#         keyword_match_Imp, score_Imp = process.extractOne(row[FINAL_COL_NAME], keywords_Imp_final, scorer = fuzz.partial_ratio)
#         row['final_score_Imp'] = score_Imp
#         row['final_keyword_match_Imp'] = keyword_match_Imp
#         return row

def fuzzy_logic(row, FINAL_COL_NAME, keywords, facet):
    keyword_match, score = process.extractOne(row[FINAL_COL_NAME], keywords, scorer = fuzz.partial_ratio)
    row['final_score_{}'.format(facet.value)] = score
    row['final_keyword_match_{}'.format(facet.value)] = keyword_match
    return row

In [185]:
def keep_only_highest(data, high_value, facet):
    data['final_score_{}'.format(facet.value)] = data['final_score_{}'.format(facet.value)].astype(int)    
    data = data[data['final_score_{}'.format(facet.value)] == 100].reset_index(drop=True)    
    return data

In [186]:
# Enum for facets
class Facets(Enum):
    IMPLEMENTATION = "imp"
    ADAPTATION = "ada"
    NEGATIVE_EMOTIONS = "ne"
    SOCIAL_DISRUPTION = "sd"
    PURPOSE = "Purp"
    POSITIVE_EMOTION = "pe"

In [187]:
# Constants
FINAL_COL_NAME = "FINAL_TEXT"

# 1_Implementation

In [188]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

In [189]:
keywords_Imp_final = clean_keywords(keywords_Imp)
#POSTags
# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

# lemmatizer = WordNetLemmatizer()
# keywords_Imp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Imp_filtered]
# keywords_Imp_final = [" ".join(sentence) for sentence in keywords_Imp_lem]

In [170]:
# data_Imp

In [190]:
data_Imp[FINAL_COL_NAME].isna().value_counts()

False    17844
True        37
Name: FINAL_TEXT, dtype: int64

In [191]:
#  def fuzzy_Imp(row):
#         keyword_match_Imp, score_Imp = process.extractOne(row[FINAL_COL_NAME], keywords_Imp_final, scorer = fuzz.partial_ratio)
#         row['final_score_Imp'] = score_Imp
#         row['final_keyword_match_Imp'] = keyword_match_Imp
#         return row

In [192]:
nan_value = float("NaN")
data_Imp[FINAL_COL_NAME].replace("", nan_value, inplace=True)

In [193]:
data_Imp.dropna(subset = [FINAL_COL_NAME], inplace=True)

In [175]:
# data_Imp


In [194]:
# interim_Imp = data_Imp.apply(fuzzy_Imp, axis=1)
# interim_Imp = fuzzy_logic(data_Imp, FINAL_COL_NAME, keywords_Imp_final, Facets.IMPLEMENTATION)
interim_Imp = data_Imp.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Imp_final, Facets.IMPLEMENTATION))

In [195]:
interim_Imp

,Unnamed: 0,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,...,QT_place_full_name,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score_imp,final_keyword_match_imp
0,11,Tue Aug 18 20:16:16 +0000 2020,1.2958168072368947e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2957219057779712e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,"Trump disses Michelle Obama: ""She was over her...","Trump disses Michelle Obama: ""She was over her...",Y,True,trump diss michel obama head frankli shouldv m...,42,stay at home
1,24,Tue Aug 18 20:16:17 +0000 2020,1.295816809204142e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.2956804607613215e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,New: Gov. Cuomo on Oct. 13 will publish a book...,NaN,Y,True,new gov cuomo oct 13 publish book the pandem “...,67,cancel
2,42,Tue Aug 18 20:16:18 +0000 2020,1.2958168136416335e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.2954302805227192e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,a girl i went to school with is panicking on I...,a girl i went to school with is panicking on I...,Y,True,girl went to school panick ig son came home fr...,62,school closur
3,56,Tue Aug 18 20:16:18 +0000 2020,1.2958168156129198e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.2945569804770796e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,500+ German doctors explaining the pandemic is...,500+ German doctors explaining the pandemic is...,Y,True,500 german doctor explain the pandem fake ever...,55,busi closur
4,69,Tue Aug 18 20:16:19 +0000 2020,1.2958168180330783e+18,"<a href=""http://twitter.com/download/android"" ...",True,1.2955132835421512e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,"방탄소년단 제이홉, 취약계층 아동 위해 1억원 기부 '총 4억5천만원'(공식)\nh...","방탄소년단 제이홉, 취약계층 아동 위해 1억원 기부 '총 4억5천만원'(공식)\nh...",Y,True,1 45 “ 19 ” jhope bt 1 45 “ 19 ” jhope bt,75,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17876,11133,Tue Aug 18 19:42:23 +0000 2020,1.2958082797807657e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2957903623432192e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,Imagina que uma pandemia atinge o nosso planet...,Imagina que uma pandemia atinge o nosso planet...,Y,True,imagina que uma pandemia ating nosso planeta e...,46,quarantinelif
17877,11136,Tue Aug 18 19:42:23 +0000 2020,1.295808279856259e+18,"<a href=""http://twitter.com/download/android"" ...",True,1.2956439387375985e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,The SNP have rejected calls to launch an inqui...,The SNP have rejected calls to launch an inqui...,Y,True,the snp reject call to launch inquiri the coro...,62,lockdown
17878,11137,Tue Aug 18 19:42:23 +0000 2020,1.295808279919186e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2957889985443144e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,No More Free Bus Rides Starting August 31st Du...,NaN,Y,True,no free bu ride start august 31st due to vast ...,60,6feet
17879,11140,Tue Aug 18 19:42:23 +0000 2020,1.2958082808081777e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.2957868673471406e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,This @Accad_Koka podcast with @mgmgomes1 on he...,This @Accad_Koka podcast with @mgmgomes1 on he...,Y,True,podcast heterogen herd immun mustlisten thread...,57,postpon


In [211]:
denominator_Imp = interim_Imp.shape[0]

In [14]:
# interim_Imp['final_score_Imp'] = interim_Imp['final_score_Imp'].astype(int)

In [212]:
# interim_Imp = interim_Imp[interim_Imp['final_score_Imp'] == 100].reset_index(drop=True)
interim_Imp = keep_only_highest(interim_Imp, 100, Facets.IMPLEMENTATION)

In [213]:
interim_Imp

,Unnamed: 0,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,...,QT_place_full_name,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score_imp,final_keyword_match_imp
0,250,Tue Aug 18 20:16:26 +0000 2020,1.2958168501446246e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.2955036779654472e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,"NEW: Mississippi has quarantined 2,035 student...","NEW: Mississippi has quarantined 2,035 student...",Y,True,new mississippi quarantin 2035 student teacher...,100,quarantin
1,348,Tue Aug 18 20:16:31 +0000 2020,1.295816869442728e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2957781171900703e+18,NaN,NaN,NaN,en,...,Bahamas,BS,NaN,"Barry Williams, 52, of Faith Gardens, said he ...","Barry Williams, 52, of Faith Gardens, said he ...",Y,True,barri william 52 faith garden said left home t...,100,home
2,390,Tue Aug 18 20:16:33 +0000 2020,1.2958168786576097e+18,"<a href=""http://twitter.com/download/android"" ...",True,1.2954036956481864e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,hey everyone 🧡 \n\ndue to positive case of cov...,hey everyone 🧡 \n\ndue to positive case of cov...,Y,True,hey everyon due to posit case covid19 workplac...,100,quarantin
3,401,Tue Aug 18 20:16:34 +0000 2020,1.2958168806037668e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2956932280968397e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,This is from the Franklin-Templeton Gallup res...,This is from the Franklin-Templeton Gallup res...,Y,True,from the franklintempleton gallup research pro...,100,respons
4,501,Tue Aug 18 20:16:38 +0000 2020,1.2958168989077217e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.2954746242019287e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,@GlennHowerton so my wife got me this. Wasn't ...,@GlennHowerton so my wife got me this. Wasn't ...,Y,True,glennhowerton wife got wasnt even custom someo...,100,social distanc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,10936,Tue Aug 18 19:42:19 +0000 2020,1.2958082613465825e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.295755341943042e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,#POSTPONEJEE_NEET \n \nLet's check Public Op...,#POSTPONEJEE_NEET \n \nLet's check Public Op...,Y,True,postponejeeneet let check public opinion do th...,100,postpon
1366,10971,Tue Aug 18 19:42:19 +0000 2020,1.2958082634648535e+18,"<a href=""http://twitter.com/download/android"" ...",True,1.2957221186389071e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,✅ Avoid large gatherings.\n✅ Maintain 6 feet b...,✅ Avoid large gatherings.\n✅ Maintain 6 feet b...,Y,True,avoid larg gather maintain 6 feet other isbt e...,100,distanc
1367,10986,Tue Aug 18 19:42:20 +0000 2020,1.295808265264214e+18,"<a href=""http://twitter.com/download/iphone"" r...",True,1.295732358214189e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,Navy Pier announced Tuesday that after Labor D...,Navy Pier announced Tuesday that after Labor D...,Y,True,navi pier announc tuesday labor day the icon t...,100,shutdown
1368,11041,Tue Aug 18 19:42:21 +0000 2020,1.2958082705530798e+18,"<a href=""http://www.echofon.com/"" rel=""nofollo...",True,1.2957804329828147e+18,NaN,NaN,NaN,en,...,NaN,NaN,NaN,Guy returns to Germany from holiday in Italy. ...,Guy returns to Germany from holiday in Italy. ...,Y,True,guy return to germani from holiday itali get f...,100,quarantin


In [221]:
numerator_Imp = interim_Imp.shape[0]

In [219]:
proportion_Imp = (numerator_Imp/denominator_Imp)
print(proportion_Imp)

1.0


# 2_Adaptation

In [19]:
keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

In [20]:
#Convert to lower
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ada)): 
    keywords_Ada[i] = keywords_Ada[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ada_tokens = [sub.split() for sub in keywords_Ada] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ada_filtered=remove_stopwords(keywords_Ada)

#Stemming
ps = PorterStemmer()
keywords_Ada_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_stem = [" ".join(sentence) for sentence in keywords_Ada_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ada_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ada_filtered]
keywords_Ada_final = [" ".join(sentence) for sentence in keywords_Ada_lem]

In [21]:
data_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [22]:
 def fuzzy_Ada(row):
        keyword_match_Ada, score_Ada = process.extractOne(row['final'], keywords_Ada_final, scorer = fuzz.partial_ratio)
        row['final_score_Ada'] = score_Ada
        row['final_keyword_match_Ada'] = keyword_match_Ada
        return row

In [23]:
nan_value = float("NaN")
data_Ada['final'].replace("", nan_value, inplace=True)

In [24]:
data_Ada.dropna(subset = ["final"], inplace=True)

In [25]:
data_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7765,1.295710e+18,1.213801e+09,NaN,MD/DC,NEW: Is your state reporting comprehensive dat...,new your state reporting comprehensive data co...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...


In [26]:
interim_Ada = data_Ada.apply(fuzzy_Ada, axis=1)

In [27]:
denominator_Ada = interim_Ada.shape[0]

In [28]:
interim_Ada['final_score_Ada'] = interim_Ada['final_score_Ada'].astype(int)

In [29]:
interim_Ada = interim_Ada[interim_Ada['final_score_Ada'] == 100].reset_index(drop=True)

In [30]:
interim_Ada

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Ada,final_keyword_match_Ada
0,1.295805e+18,1.466334e+07,NaN,New York,"Antonio Argibay, AIA, managing principal of Me...",antonio argibay aia manage principal meridian ...,100,wfh
1,1.295805e+18,2.994861e+09,NaN,NaN,@PJS412 @Erinmc33 Patrick knows where I work. ...,patrick know work early pandemic work from hom...,100,work from home
2,1.295805e+18,9.889688e+08,NaN,NaN,@JohnWil91460279 The crazies are protesting t...,the crazy protest to vote mail say fear to go...,100,to go
3,1.295805e+18,2.913471e+09,NaN,"Omaha, Nebraska",The worst part about this remote learning is t...,the bad part remote learn strip to underwear ...,100,learn
4,1.295805e+18,4.125046e+08,NaN,"Avon, CT",An injection of the #flu vaccine will not give...,injection the flu vaccine give the flu learn,100,learn
...,...,...,...,...,...,...,...,...
274,1.295773e+18,1.590615e+08,NaN,Clydesdale,"Today I spoke to one family - Mum, dad &amp; t...",today spoke to one family mum dad amp lass cha...,100,learn
275,1.295525e+18,2.736106e+09,NaN,Bahamas,I still think delivery options would help. Del...,still think delivery option would help deliver...,100,delivery
276,1.295781e+18,1.295041e+18,NaN,Western Cape,I HAVE LEARNT.\n\nI have learnt :-\nthat you c...,learnt learnt make someone love do someone lov...,100,learn
277,1.295763e+18,1.900636e+08,NaN,"Washington, D.C.",Today’s first @washingtonpost quarantine TikTo...,today first quarantine tiktok feature the sta...,100,tiktok


In [31]:
numerator_Ada = interim_Ada.shape[0]

In [32]:
proportion_Ada = (numerator_Ada/denominator_Ada)
print(proportion_Ada)

0.03597678916827853


# 3_Negative Emotion

In [33]:
keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

In [34]:
#Convert to lower
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].lower()

#Remove punctuations   
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Ne)): 
    keywords_Ne[i] = keywords_Ne[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Ne_tokens = [sub.split() for sub in keywords_Ne] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Ne_filtered=remove_stopwords(keywords_Ne)

#Stemming
ps = PorterStemmer()
keywords_Ne_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_stem = [" ".join(sentence) for sentence in keywords_Ne_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Ne_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Ne_filtered]
keywords_Ne_final = [" ".join(sentence) for sentence in keywords_Ne_lem]

In [35]:
data_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [36]:
 def fuzzy_Ne(row):
        keyword_match_Ne, score_Ne = process.extractOne(row['final'], keywords_Ne_final, scorer = fuzz.partial_ratio)
        row['final_score_Ne'] = score_Ne
        row['final_keyword_match_Ne'] = keyword_match_Ne
        return row

In [37]:
nan_value = float("NaN")
data_Ne['final'].replace("", nan_value, inplace=True)

In [38]:
data_Ne.dropna(subset = ["final"], inplace=True)

In [39]:
data_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7765,1.295710e+18,1.213801e+09,NaN,MD/DC,NEW: Is your state reporting comprehensive dat...,new your state reporting comprehensive data co...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...


In [40]:
interim_Ne = data_Ne.apply(fuzzy_Ne, axis=1)

In [41]:
denominator_Ne = interim_Ne.shape[0]

In [42]:
interim_Ne['final_score_Ne'] = interim_Ne['final_score_Ne'].astype(int)

In [43]:
interim_Ne = interim_Ne[interim_Ne['final_score_Ne'] == 100].reset_index(drop=True)

In [44]:
interim_Ne

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Ne,final_keyword_match_Ne
0,1.295805e+18,2.232332e+07,NaN,Nairobi/Kyiv/Nassau/Kosovo,"Hey, friends. . . I know a lot of y'all #homes...",hey friend know lot yall homeschool supportive...,100,end
1,1.295805e+18,7.136334e+07,NaN,NaN,#VIU prof. Dr. Aggie Weighill needs your input...,viu prof dr aggie weighill need your input can...,100,stress
2,1.295805e+18,7.219786e+08,NaN,NaN,Why is ASUU trending? \n\nAre they still on st...,asuu trend still strike theyre also demand cov...,100,end
3,1.295805e+18,1.644156e+07,NaN,"Oklahoma, USA",Quote of the Day from State Superintendent Joy...,quote the day from state superintendent joy ho...,100,end
4,1.295805e+18,1.569142e+08,NaN,"Carmel, IN","No matter what your political views are, as ha...",no matter your political view hard may set asi...,100,end
...,...,...,...,...,...,...,...,...
386,1.295080e+18,3.827117e+08,NaN,"Massachusetts, USA",Yesterday I got to see my best friend for the ...,yesterday get to see best friend the first tim...,100,end
387,1.295805e+18,4.728833e+08,NaN,Devon,@phoenixgrouphq @PhoenixEdSarah @DrBenCostello...,worry the value grade see educator employer th...,100,worry
388,1.295693e+18,1.446385e+07,NaN,"Washington, DC","In just months, #COVID19 has set back a decade...",month covid19 set back decade progress the bat...,100,end
389,1.295730e+18,3.287109e+07,NaN,"Manhattan, NY","AARP to the Postmaster General: ""We urge you t...",aarp to the postmaster general urge to suspend...,100,end


In [45]:
numerator_Ne = interim_Ne.shape[0]

In [46]:
proportion_Ne = (numerator_Ne/denominator_Ne)
print(proportion_Ne)

0.050419084461637655


# 4_Social Disruption

In [47]:
keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd',]

In [48]:
#Convert to lower
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].lower()

#Remove punctuations   
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Sd)): 
    keywords_Sd[i] = keywords_Sd[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Sd_tokens = [sub.split() for sub in keywords_Sd] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Sd_filtered=remove_stopwords(keywords_Sd)

#Stemming
ps = PorterStemmer()
keywords_Sd_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_stem = [" ".join(sentence) for sentence in keywords_Sd_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Sd_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Sd_filtered]
keywords_Sd_final = [" ".join(sentence) for sentence in keywords_Sd_lem]

In [49]:
data_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [50]:
 def fuzzy_Sd(row):
        keyword_match_Sd, score_Sd = process.extractOne(row['final'], keywords_Sd_final, scorer = fuzz.partial_ratio)
        row['final_score_Sd'] = score_Sd
        row['final_keyword_match_Sd'] = keyword_match_Sd
        return row

In [51]:
nan_value = float("NaN")
data_Sd['final'].replace("", nan_value, inplace=True)

In [52]:
data_Sd.dropna(subset = ["final"], inplace=True)

In [53]:
data_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7765,1.295710e+18,1.213801e+09,NaN,MD/DC,NEW: Is your state reporting comprehensive dat...,new your state reporting comprehensive data co...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...


In [54]:
interim_Sd = data_Sd.apply(fuzzy_Sd, axis=1)

In [55]:
denominator_Sd = interim_Sd.shape[0]

In [56]:
interim_Sd['final_score_Sd'] = interim_Sd['final_score_Sd'].astype(int)

In [57]:
interim_Sd = interim_Sd[interim_Sd['final_score_Sd'] == 100].reset_index(drop=True)

In [58]:
interim_Sd

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Sd,final_keyword_match_Sd
0,1.295805e+18,2.231906e+09,NaN,Istanbul,"Norway’s sovereign wealth fund, the world’s la...",norway sovereign wealth fund the world large...,100,large
1,1.295805e+18,1.269774e+18,NaN,Oxford,@KarlTurnerMP Social distancing rules have bee...,social distance rule largely ignore since earl...,100,large
2,1.295805e+18,9.822593e+17,NaN,"Pennsylvania, USA",KOMMUNISTS KONTROL of FOOD AND TRAVEL Peggy ...,kommunists kontrol food travel peggy hall via ...,100,travel
3,1.295805e+18,1.295469e+18,NaN,NaN,Guys today i wrote my upcatet examination and ...,guy today write upcatet examination let tell o...,100,crowd
4,1.295805e+18,8.159179e+07,NaN,"New York, NY",https://t.co/M6WudONqY1 @UMassAmherst are offe...,offering wifibased contact trace app to school...,100,interaction
...,...,...,...,...,...,...,...,...
162,1.295739e+18,8.309835e+17,NaN,Northern Virginia,People at the Farmville Detention Center are e...,people at the farmville detention center exper...,100,large
163,1.295082e+18,3.391985e+09,NaN,NaN,"Large crowds gathered yesterday In Ilford, Eas...",large crowd gather yesterday ilford east londo...,100,large
164,1.295364e+18,4.878335e+08,NaN,London,A paper in Nature reveals the structures of th...,paper nature reveals the structure the sarscov...,100,interaction
165,1.295752e+18,1.425158e+09,NaN,Worldwide,The ISN staff is joining the @WHO #WearAMask c...,the staff join the wearamask challenge invite ...,100,crowd


In [59]:
numerator_Sd = interim_Sd.shape[0]

In [60]:
proportion_Sd = (numerator_Sd/denominator_Sd)
print(proportion_Sd)

0.021534493874919406


# 5_Purpose

In [61]:
keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

In [62]:
#Convert to lower
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].lower()

#Remove punctuations   
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Purp)): 
    keywords_Purp[i] = keywords_Purp[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Purp_tokens = [sub.split() for sub in keywords_Purp] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Purp_filtered=remove_stopwords(keywords_Purp)

#Stemming
ps = PorterStemmer()
keywords_Purp_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_stem = [" ".join(sentence) for sentence in keywords_Purp_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Purp_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Purp_filtered]
keywords_Purp_final = [" ".join(sentence) for sentence in keywords_Purp_lem]

In [63]:
data_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [64]:
 def fuzzy_Purp(row):
        keyword_match_Purp, score_Purp = process.extractOne(row['final'], keywords_Purp_final, scorer = fuzz.partial_ratio)
        row['final_score_Purp'] = score_Purp
        row['final_keyword_match_Purp'] = keyword_match_Purp
        return row

In [65]:
nan_value = float("NaN")
data_Purp['final'].replace("", nan_value, inplace=True)

In [66]:
data_Purp.dropna(subset = ["final"], inplace=True)

In [67]:
data_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7765,1.295710e+18,1.213801e+09,NaN,MD/DC,NEW: Is your state reporting comprehensive dat...,new your state reporting comprehensive data co...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...


In [68]:
interim_Purp = data_Purp.apply(fuzzy_Purp, axis=1)

In [69]:
denominator_Purp = interim_Purp.shape[0]

In [70]:
interim_Purp['final_score_Purp'] = interim_Purp['final_score_Purp'].astype(int)

In [71]:
interim_Purp = interim_Purp[interim_Purp['final_score_Purp'] == 100].reset_index(drop=True)

In [72]:
interim_Purp

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Purp,final_keyword_match_Purp
0,1.295805e+18,1.212085e+18,NaN,NaN,#Checkout this #new #facemask #fineartamerica ...,checkout new facemask fineartamerica covid19 p...,100,protect
1,1.295805e+18,1.001892e+18,NaN,NaN,@marcdaalder I think we carve our own destiny....,think carve destiny build quarantine facility ...,100,save
2,1.295805e+18,2.396240e+07,NaN,Straight on 'til morning.,@TheBev_Seattle @BobFergusonAG I noticed the w...,notice the winery pandemic id love to visit th...,100,save
3,1.295805e+18,8.699340e+08,NaN,"Chariton, IA","Our Surgery Team (Dr. Rauen, Jennie Hall, and ...",surgery team dr rauen jennie hall konrad konsi...,100,protect
4,1.295805e+18,1.425606e+09,NaN,"North Carolina, USA",Following the Three Ws is a great way to #Stop...,follow the three w great way to stopthespread ...,100,protect
...,...,...,...,...,...,...,...,...
168,1.295767e+18,1.982584e+07,NaN,London & Suffolk,The first responsibility of any Government is ...,the first responsibility government to protect...,100,protect
169,1.293072e+18,1.070941e+18,NaN,NaN,"#SaveTheChildren \nThere are over 800,000 chil...",savethechildren 800000 child miss the u alone ...,100,save
170,1.295699e+18,2.575115e+09,NaN,South Africa,Kebby Maphatsoe (MKMVA Pres):\n\nWhen it suite...,kebby maphatsoe mkmva pres suit savesa 101 vet...,100,save
171,1.294734e+18,2.277613e+07,NaN,"Westminster, London",EXCLUSIVE Sunday @Telegraph splash\nPublic Hea...,exclusive sunday splash public health england ...,100,protect


In [73]:
numerator_Purp = interim_Purp.shape[0]

In [74]:
proportion_Purp = (numerator_Purp/denominator_Purp)
print(proportion_Purp)

0.022308188265635073


# 6_Positive Emotion

In [75]:
keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [76]:
#Convert to lower
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].lower()

#Remove punctuations   
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords_Pe)): 
    keywords_Pe[i] = keywords_Pe[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_Pe_tokens = [sub.split() for sub in keywords_Pe] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_Pe_filtered=remove_stopwords(keywords_Pe)

#Stemming
ps = PorterStemmer()
keywords_Pe_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_stem = [" ".join(sentence) for sentence in keywords_Pe_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_Pe_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_Pe_filtered]
keywords_Pe_final = [" ".join(sentence) for sentence in keywords_Pe_lem]

In [77]:
data_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [78]:
 def fuzzy_Pe(row):
        keyword_match_Pe, score_Pe = process.extractOne(row['final'], keywords_Pe_final, scorer = fuzz.partial_ratio)
        row['final_score_Pe'] = score_Pe
        row['final_keyword_match_Pe'] = keyword_match_Pe
        return row

In [79]:
nan_value = float("NaN")
data_Pe['final'].replace("", nan_value, inplace=True)

In [80]:
data_Pe.dropna(subset = ["final"], inplace=True)

In [81]:
data_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7765,1.295710e+18,1.213801e+09,NaN,MD/DC,NEW: Is your state reporting comprehensive dat...,new your state reporting comprehensive data co...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...


In [82]:
interim_Pe = data_Pe.apply(fuzzy_Pe, axis=1)

In [83]:
denominator_Pe = interim_Pe.shape[0]

In [84]:
interim_Pe['final_score_Pe'] = interim_Pe['final_score_Pe'].astype(int)

In [85]:
interim_Pe = interim_Pe[interim_Pe['final_score_Pe'] == 100].reset_index(drop=True)

In [86]:
interim_Pe

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final,final_score_Pe,final_keyword_match_Pe
0,1.295805e+18,1.170410e+18,NaN,Paris,Thank you also @GradientDragon for what turned...,thank also turn out to lovely heartfelt aftern...,100,hope
1,1.295805e+18,1.268273e+18,NaN,NaN,Our resident biology major @danie1mk says ther...,resident biology major say no hope fall footba...,100,hope
2,1.295805e+18,1.212085e+18,NaN,NaN,#Checkout this #new #facemask #fineartamerica ...,checkout new facemask fineartamerica covid19 p...,100,safe
3,1.295805e+18,1.967207e+07,NaN,"Dallas, Texas",We were so excited to host an event over the w...,excite to host event the weekend complete soci...,100,safe
4,1.295805e+18,2.693147e+09,NaN,NaN,Keeping Your Crew Safe During COVID-19...https...,keep your crew safe covid19,100,safe
...,...,...,...,...,...,...,...,...
237,1.290330e+18,2.987672e+09,NaN,"Annapolis, MD",I have issued an amended emergency order ensur...,issue amend emergency order ensure local schoo...,100,safe
238,1.295682e+18,1.647291e+09,NaN,"West Midlands, England",2064: Year 7’s return to school after the Teac...,2064 year 7 return to school the teach union ...,100,safe
239,1.294698e+18,5.614415e+08,NaN,NaN,Kudos to @Walmart for discovering that the vir...,kudos to discover the virus dormant 829 pm go ...,100,safe
240,1.294303e+18,1.615356e+07,NaN,New York,".@DHS_Wolf &amp; @ICEgov, you heard 'em! \n\nF...",amp heard em family safeandtogether freedom de...,100,safe


In [87]:
numerator_Pe = interim_Pe.shape[0]

In [88]:
proportion_Pe = (numerator_Pe/denominator_Pe)
print(proportion_Pe)

0.031205673758865248


# All Proportions

In [89]:
print(proportion_Imp)

0.17421018697614443


In [90]:
print(proportion_Ada)

0.03597678916827853


In [91]:
print(proportion_Ne)

0.050419084461637655


In [92]:
print(proportion_Sd)

0.021534493874919406


In [93]:
print(proportion_Purp)

0.022308188265635073


In [94]:
print(proportion_Pe)

0.031205673758865248


# Checking for same Tweet in different Facet (15 Cases)

### 1_Implementation and 2_Adaptation

In [137]:
Imp_Ada = pd.merge(interim_Imp, interim_Ada, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [138]:
Imp_Ada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 0 to 104
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    105 non-null    float64
 1   user_id_x                             105 non-null    float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            76 non-null     object 
 4   extended_tweet_full_text_duplicate_x  105 non-null    object 
 5   final_x                               105 non-null    object 
 6   final_score_Imp                       105 non-null    int64  
 7   final_keyword_match_Imp               105 non-null    object 
 8   user_id_y                             105 non-null    float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            76 non-null     object 
 11  extended_tweet_full

### 1_Implementation and 3_Negative Emotion

In [97]:
Imp_Ne = pd.merge(interim_Imp, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [98]:
Imp_Ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    75 non-null     float64
 1   user_id_x                             75 non-null     float64
 2   coordinates_x                         1 non-null      object 
 3   location_x                            63 non-null     object 
 4   extended_tweet_full_text_duplicate_x  75 non-null     object 
 5   final_x                               75 non-null     object 
 6   final_score_Imp                       75 non-null     int64  
 7   final_keyword_match_Imp               75 non-null     object 
 8   user_id_y                             75 non-null     float64
 9   coordinates_y                         1 non-null      object 
 10  location_y                            63 non-null     object 
 11  extended_tweet_full_t

### 1_Implementation and 4_Social Disruption

In [99]:
Imp_Sd = pd.merge(interim_Imp, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [100]:
Imp_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    60 non-null     float64
 1   user_id_x                             60 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            52 non-null     object 
 4   extended_tweet_full_text_duplicate_x  60 non-null     object 
 5   final_x                               60 non-null     object 
 6   final_score_Imp                       60 non-null     int64  
 7   final_keyword_match_Imp               60 non-null     object 
 8   user_id_y                             60 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            52 non-null     object 
 11  extended_tweet_full_t

### 1_Implementation and 5_Purpose

In [101]:
Imp_Purp = pd.merge(interim_Imp, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [102]:
Imp_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    25 non-null     float64
 1   user_id_x                             25 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            21 non-null     object 
 4   extended_tweet_full_text_duplicate_x  25 non-null     object 
 5   final_x                               25 non-null     object 
 6   final_score_Imp                       25 non-null     int64  
 7   final_keyword_match_Imp               25 non-null     object 
 8   user_id_y                             25 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            21 non-null     object 
 11  extended_tweet_full_t

### 1_Implementation and 6_Positive Emotion

In [103]:
Imp_Pe = pd.merge(interim_Imp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [104]:
Imp_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    49 non-null     float64
 1   user_id_x                             49 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            37 non-null     object 
 4   extended_tweet_full_text_duplicate_x  49 non-null     object 
 5   final_x                               49 non-null     object 
 6   final_score_Imp                       49 non-null     int64  
 7   final_keyword_match_Imp               49 non-null     object 
 8   user_id_y                             49 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            37 non-null     object 
 11  extended_tweet_full_t

### 2_Adaptation and 3_Negative Emotion

In [105]:
Ada_Ne = pd.merge(interim_Ada, interim_Ne, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [106]:
Ada_Ne.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    12 non-null     float64
 1   user_id_x                             12 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            11 non-null     object 
 4   extended_tweet_full_text_duplicate_x  12 non-null     object 
 5   final_x                               12 non-null     object 
 6   final_score_Ada                       12 non-null     int64  
 7   final_keyword_match_Ada               12 non-null     object 
 8   user_id_y                             12 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            11 non-null     object 
 11  extended_tweet_full_t

### 2_Adaptation and 4_Social Disruption

In [107]:
Ada_Sd = pd.merge(interim_Ada, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [108]:
Ada_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 7
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    8 non-null      float64
 1   user_id_x                             8 non-null      float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            7 non-null      object 
 4   extended_tweet_full_text_duplicate_x  8 non-null      object 
 5   final_x                               8 non-null      object 
 6   final_score_Ada                       8 non-null      int64  
 7   final_keyword_match_Ada               8 non-null      object 
 8   user_id_y                             8 non-null      float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            7 non-null      object 
 11  extended_tweet_full_tex

### 2_Adaptation and 5_Purpose

In [109]:
Ada_Purp = pd.merge(interim_Ada, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [110]:
Ada_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    14 non-null     float64
 1   user_id_x                             14 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            11 non-null     object 
 4   extended_tweet_full_text_duplicate_x  14 non-null     object 
 5   final_x                               14 non-null     object 
 6   final_score_Ada                       14 non-null     int64  
 7   final_keyword_match_Ada               14 non-null     object 
 8   user_id_y                             14 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            11 non-null     object 
 11  extended_tweet_full_t

### 2_Adaptation and 6_Positive Emotion

In [111]:
Ada_Pe = pd.merge(interim_Ada, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [112]:
Ada_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    11 non-null     float64
 1   user_id_x                             11 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            7 non-null      object 
 4   extended_tweet_full_text_duplicate_x  11 non-null     object 
 5   final_x                               11 non-null     object 
 6   final_score_Ada                       11 non-null     int64  
 7   final_keyword_match_Ada               11 non-null     object 
 8   user_id_y                             11 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            7 non-null      object 
 11  extended_tweet_full_t

### 3_Negative Emotion and 4_Social Disruption

In [113]:
Ne_Sd = pd.merge(interim_Ne, interim_Sd, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [114]:
Ne_Sd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    12 non-null     float64
 1   user_id_x                             12 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            11 non-null     object 
 4   extended_tweet_full_text_duplicate_x  12 non-null     object 
 5   final_x                               12 non-null     object 
 6   final_score_Ne                        12 non-null     int64  
 7   final_keyword_match_Ne                12 non-null     object 
 8   user_id_y                             12 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            11 non-null     object 
 11  extended_tweet_full_t

### 3_Negative Emotion and 5_Purpose

In [115]:
Ne_Purp = pd.merge(interim_Ne, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [116]:
Ne_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6 non-null      float64
 1   user_id_x                             6 non-null      float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            6 non-null      object 
 4   extended_tweet_full_text_duplicate_x  6 non-null      object 
 5   final_x                               6 non-null      object 
 6   final_score_Ne                        6 non-null      int64  
 7   final_keyword_match_Ne                6 non-null      object 
 8   user_id_y                             6 non-null      float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            6 non-null      object 
 11  extended_tweet_full_tex

### 3_Negative Emotion and 6_Positive Emotion

In [117]:
Ne_Pe = pd.merge(interim_Ne, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [118]:
Ne_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    14 non-null     float64
 1   user_id_x                             14 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            11 non-null     object 
 4   extended_tweet_full_text_duplicate_x  14 non-null     object 
 5   final_x                               14 non-null     object 
 6   final_score_Ne                        14 non-null     int64  
 7   final_keyword_match_Ne                14 non-null     object 
 8   user_id_y                             14 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            11 non-null     object 
 11  extended_tweet_full_t

### 4_Social Disruption and 5_Purpose

In [119]:
Sd_Purp = pd.merge(interim_Sd, interim_Purp, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [120]:
Sd_Purp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    3 non-null      float64
 1   user_id_x                             3 non-null      float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            3 non-null      object 
 4   extended_tweet_full_text_duplicate_x  3 non-null      object 
 5   final_x                               3 non-null      object 
 6   final_score_Sd                        3 non-null      int64  
 7   final_keyword_match_Sd                3 non-null      object 
 8   user_id_y                             3 non-null      float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            3 non-null      object 
 11  extended_tweet_full_tex

### 4_Social Disruption and 6_Positive Emotion

In [121]:
Sd_Pe = pd.merge(interim_Sd, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [122]:
Sd_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    11 non-null     float64
 1   user_id_x                             11 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            10 non-null     object 
 4   extended_tweet_full_text_duplicate_x  11 non-null     object 
 5   final_x                               11 non-null     object 
 6   final_score_Sd                        11 non-null     int64  
 7   final_keyword_match_Sd                11 non-null     object 
 8   user_id_y                             11 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            10 non-null     object 
 11  extended_tweet_full_t

### 5_Purpose and 6_Positive Emotion

In [123]:
Purp_Pe = pd.merge(interim_Purp, interim_Pe, how = 'inner', on = ['id'], left_index=False, right_index=False, sort=True)

In [124]:
Purp_Pe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    11 non-null     float64
 1   user_id_x                             11 non-null     float64
 2   coordinates_x                         0 non-null      object 
 3   location_x                            10 non-null     object 
 4   extended_tweet_full_text_duplicate_x  11 non-null     object 
 5   final_x                               11 non-null     object 
 6   final_score_Purp                      11 non-null     int64  
 7   final_keyword_match_Purp              11 non-null     object 
 8   user_id_y                             11 non-null     float64
 9   coordinates_y                         0 non-null      object 
 10  location_y                            10 non-null     object 
 11  extended_tweet_full_t